In [9]:
import sqlite3
import pandas as pd
import numpy as np

# 1. Підключення до бази даних
conn = sqlite3.connect("C:/Databases/charge_database_alt.sqlite")
# Завантаження таблиць
df = pd.read_sql_query("SELECT * FROM ChargingSessions_Encoded;", conn)
df_Vehicles = pd.read_sql_query("SELECT * FROM Vehicles;", conn)
print(df_Vehicles.columns)

Index(['vehicle_id', 'user_id', 'vehicle_model', 'battery_capacity_kwh',
       'vehicle_age_years'],
      dtype='object')


In [10]:
# Список числових колонок
cols = ['charging_duration_hours','charging_cost_usd','temperature_c','state_of_charge_start','state_of_charge_end','energy_consumed_kwh', 'charging_rate_kw', 'distance_driven_km']

# Перевіримо описову статистику
print(df[cols].describe())

# Виявлення потенційних аномалій методом IQR (Interquartile Range)
for col in cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    median = df[col].median()
    df.loc[(df[col] < lower_bound) | (df[col] > upper_bound), col] = median
# --- додаткова логічна перевірка для реалістичних меж ---
    if col == 'temperature_c':
        df.loc[(df[col] < -20) | (df[col] > 45), col] = median
    if col == 'state_of_charge_start':
        df.loc[(df[col] < 0) | (df[col] > 100), col] = median
    if col == 'state_of_charge_end':
        df.loc[(df[col] < 0) | (df[col] > 100), col] = median

       charging_duration_hours  charging_cost_usd  temperature_c  \
count              1320.000000        1320.000000    1320.000000   
mean                  2.269377          22.551352      15.263591   
std                   1.061037          10.751494      14.831216   
min                   0.095314           0.234317     -10.724770   
25%                   1.397623          13.368141       2.800664   
50%                   2.258136          22.076360      14.630846   
75%                   3.112806          31.646044      27.981810   
max                   7.635145          69.407743      73.169588   

       state_of_charge_start  state_of_charge_end  energy_consumed_kwh  \
count            1320.000000          1320.000000          1320.000000   
mean               49.130012            75.141590            42.645320   
std                24.074134            17.080580            21.843794   
min                 2.325959             7.604224             0.045772   
25%              

In [11]:
# Список числових колонок
col = 'battery_capacity_kwh'

# Перевіримо описову статистику
print(df_Vehicles['battery_capacity_kwh'].describe())

# Виявлення потенційних аномалій методом IQR (Interquartile Range)
Q1 = df_Vehicles['battery_capacity_kwh'].quantile(0.25)
Q3 = df_Vehicles['battery_capacity_kwh'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
median = df_Vehicles['battery_capacity_kwh'].median()
df_Vehicles.loc[(df_Vehicles['battery_capacity_kwh'] < lower_bound) | (df_Vehicles['battery_capacity_kwh'] > upper_bound), 'battery_capacity_kwh'] = median

count    1320.000000
mean       74.534692
std        20.626914
min         1.532807
25%        62.000000
50%        75.000000
75%        85.000000
max       193.003074
Name: battery_capacity_kwh, dtype: float64


In [16]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# Вибір методу
scaler = StandardScaler()

In [14]:
# Список числових колонок
cols = ['charging_duration_hours','charging_cost_usd','temperature_c','state_of_charge_start','state_of_charge_end','energy_consumed_kwh', 'charging_rate_kw', 'distance_driven_km']
df[cols] = scaler.fit_transform(df[cols])
cols = ['battery_capacity_kwh']
df_Vehicles[cols] = scaler.fit_transform(df_Vehicles[cols])

In [15]:
df.to_sql("ChargingSessions_ScaledNoAnomaly", conn, if_exists="replace", index=False)
df_Vehicles.to_sql("Vehicles_ScaledNoAnomaly", conn, if_exists="replace", index=False)
conn.close()